<a href="https://colab.research.google.com/github/Ivan8Garcia/data-science/blob/main/Pandas_Tratamiento_Manipulacion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Problema del negocio**

se trata de un algoritmo capaz de analizar las caracteristicas  de una propiedad -como comodidades,tamaño, ocupacion del espacio en un  periodo determinado- y sugerir al anfitrion un precio a cobrar por tarifas diarias que garantice ganancias en momentos de alta demanda

##**1. Entendiendo el problema**

In [1]:
import pandas as pd

In [2]:
datos= pd.read_json("/content/datos_hosting.json")
datos.head()

,info_inmuebles
0,"{'evaluacion_general': '10.0', 'experiencia_lo..."
1,"{'evaluacion_general': '10.0', 'experiencia_lo..."
2,"{'evaluacion_general': '10.0', 'experiencia_lo..."
3,"{'evaluacion_general': '10.0', 'experiencia_lo..."
4,"{'evaluacion_general': '10.0', 'experiencia_lo..."


In [3]:
#usamos NORMALIZE para convertir el archivo JSON en un archivo normal,y nos lo devuelve siempre como un dataframe
#el primer "datos" es para el nuevo dataframe generado y el segundo "datos" es para especificar la primer variable que usamos para la ruta
# y se estará sobrescribiendo la primer variable datos porque ahora contendrá un data frame con todas las columnas del diccionario no abierto
datos= pd.json_normalize(datos["info_inmuebles"])
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,[This clean and comfortable one bedroom sits r...,[Lower Queen Anne is near the Seattle Center (...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Real Bed, Futon, Futon, Pull-out Sofa, Real B...","[{Internet,""Wireless Internet"",Kitchen,""Free P...","[$0, $0, $0, $0, $0, $350.00, $350.00, $350.00...","[$0, $0, $0, $20.00, $15.00, $28.00, $35.00, $...","[$110.00, $45.00, $55.00, $52.00, $85.00, $50...."
1,10.0,--,10,[Welcome to the heart of the 'Ballard Brewery ...,"[--, Capital Hill is the heart of Seattle, bor...","[2, 3, 2, 3, 3, 3, 2, 1, 2, 2, 2]","[3, 4, 2, 3, 3, 3, 3, 3, 3, 4, 3]","[5, 6, 8, 3, 3, 5, 4, 5, 6, 7, 4]","[Real Bed, Real Bed, Real Bed, Real Bed, Real ...","[{TV,Internet,""Wireless Internet"",Kitchen,""Fre...","[$500.00, $300.00, $0, $300.00, $300.00, $360....","[$125.00, $100.00, $85.00, $110.00, $110.00, $...","[$350.00, $300.00, $425.00, $300.00, $285.00, ..."
2,10.0,--,11,[New modern house built in 2013. Spectacular ...,[Upper Queen Anne is a charming neighborhood f...,[4],[5],[7],[Real Bed],"[{TV,""Cable TV"",Internet,""Wireless Internet"",""...","[$1,000.00]",[$300.00],[$975.00]
3,10.0,--,12,[Our NW style home is 3200+ sq ft with 3 level...,[The Views from our top floor! Wallingford ha...,"[3, 3, 3, 3, 3, 3, 3, 3]","[6, 6, 5, 5, 5, 5, 4, 4]","[6, 6, 7, 8, 7, 7, 6, 6]","[Real Bed, Real Bed, Real Bed, Real Bed, Real ...","[{Internet,""Wireless Internet"",Kitchen,""Free P...","[$500.00, $500.00, $500.00, $500.00, $500.00, ...","[$225.00, $300.00, $250.00, $250.00, $250.00, ...","[$490.00, $550.00, $350.00, $350.00, $350.00, ..."
4,10.0,--,14,"[Perfect for groups. 2 bedrooms, full bathroom...",[Safeway grocery store within walking distance...,"[2, 3]","[2, 6]","[3, 9]","[Real Bed, Real Bed]","[{TV,Internet,""Wireless Internet"",Kitchen,""Fre...","[$300.00, $2,000.00]","[$40.00, $150.00]","[$200.00, $545.00]"


##**2. Datos Numericos**

In [4]:
columnas= list(datos.columns)
columnas

['evaluacion_general',
 'experiencia_local',
 'max_hospedes',
 'descripcion_local',
 'descripcion_vecindad',
 'cantidad_baños',
 'cantidad_cuartos',
 'cantidad_camas',
 'modelo_cama',
 'comodidades',
 'cuota_deposito',
 'cuota_limpieza',
 'precio']

In [5]:
datos= datos.explode(columnas[3:])
datos

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$0,$110.00
0,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$45.00
0,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$55.00
0,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$20.00,$52.00
0,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",$0,$15.00,$85.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,nan,--,8,Beautiful craftsman home in the historic Wedgw...,--,3,4,5,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...","$1,000.00",$178.00,$299.00
68,nan,--,8,Located in a very easily accessible area of Se...,"Quiet, dead end street near I-5. The proximity...",2,4,4,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",$0,$99.00,$199.00
68,nan,--,8,This home is fully furnished and available wee...,--,1,3,4,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",$0,$0,$400.00
69,nan,--,9,This business-themed modern home features: *H...,Your hosts made Madison Valley their home when...,2,3,6,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...","$1,000.00",$150.00,$250.00


In [6]:
#METODO PARA RESETEAR EL INDICE, ya que en la celda anterior nos aparecen muchas veces los mismos numeros en el indice
#primero se invoca al df datos, inplace es para actualizar en el mismo df  y drop es para borrar de la memoria el viejo indice
datos.reset_index(inplace= True, drop= True)
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$0,$110.00
1,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$45.00
2,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",$0,$0,$55.00
3,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",$0,$20.00,$52.00
4,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",$0,$15.00,$85.00


In [7]:
#el tipo de datos OBJECT aparece cuando no sabe si es texto o numero; por lo tanto está mal y hay que seguir tratando los datos
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   object
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   object
 6   cantidad_cuartos      3818 non-null   object
 7   cantidad_camas        3818 non-null   object
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: object(13)
memory usage: 387.9+ KB


In [8]:
import numpy as np

In [9]:
datos["max_hospedes"]=datos["max_hospedes"].astype(np.int64)


In [10]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   int64 
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   object
 6   cantidad_cuartos      3818 non-null   object
 7   cantidad_camas        3818 non-null   object
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: int64(1), object(12)
memory usage: 387.9+ KB


In [11]:
#haciendo una lista de las columnas a cambiar como tipo int64 y guardandola en una variable para hacerlo mas rapido
col_numericas= ["cantidad_baños","cantidad_cuartos","cantidad_camas"]

In [12]:
datos[col_numericas]=datos[col_numericas].astype(np.int64)

In [13]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   evaluacion_general    3818 non-null   object
 1   experiencia_local     3818 non-null   object
 2   max_hospedes          3818 non-null   int64 
 3   descripcion_local     3818 non-null   object
 4   descripcion_vecindad  3818 non-null   object
 5   cantidad_baños        3818 non-null   int64 
 6   cantidad_cuartos      3818 non-null   int64 
 7   cantidad_camas        3818 non-null   int64 
 8   modelo_cama           3818 non-null   object
 9   comodidades           3818 non-null   object
 10  cuota_deposito        3818 non-null   object
 11  cuota_limpieza        3818 non-null   object
 12  precio                3818 non-null   object
dtypes: int64(4), object(9)
memory usage: 387.9+ KB


In [14]:
datos["evaluacion_general"]=datos["evaluacion_general"].astype(np.float64)

In [15]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   object 
 11  cuota_limpieza        3818 non-null   object 
 12  precio                3818 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 387.9+ KB


In [16]:
datos["precio"]

,precio
0,$110.00
1,$45.00
2,$55.00
3,$52.00
4,$85.00
...,...
3813,$299.00
3814,$199.00
3815,$400.00
3816,$250.00


In [17]:
#tranformando la columna precio para que tengamos los valores adecuados en datos.info(), "aplicando un lamba x:"" con el metodo replace
#dentro del primer parentesis se sustituye el signo dolar por nada y en el segundo parentesis se sustituyen las posibles comas existentes por nada
# y el strip se usa para remover los espacios y solamente quede el numero
datos["precio"]=datos["precio"].apply(lambda x: x.replace("$","").replace(",","").strip())


In [18]:
#vamos a la columna precio para verificar que se haya hecho el cambio de object a float y vemos que no pasó nadaporque no hemos hecho el astype
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   object 
 11  cuota_limpieza        3818 non-null   object 
 12  precio                3818 non-null   object 
dtypes: float64(1), int64(4), object(8)
memory usage: 387.9+ KB


In [19]:
#ahora si se aplica el cambio y se almacena en el df datos de la columna precio y lo confirmamos con un datos.info()
datos["precio"]= datos["precio"].astype(np.float64)

In [20]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   object 
 11  cuota_limpieza        3818 non-null   object 
 12  precio                3818 non-null   float64
dtypes: float64(2), int64(4), object(7)
memory usage: 387.9+ KB


In [21]:
datos["precio"].head()

,precio
0,110.0
1,45.0
2,55.0
3,52.0
4,85.0


In [23]:
#cuando es una sola columna se lee como serie y no es dataframe y por eso se usa .apply
datos[["cuota_deposito","cuota_limpieza"]].apply(lambda x: x.replace("$","").replace(",","").strip())

AttributeError: 'Series' object has no attribute 'strip'

In [24]:
#por ultimo cuando inspeccionamos mas de una columna se utiliza doble parentesis
#aqui se usa apply.map cuando son de dos columnas en adelante y porque tambien es un dataframe
#cuando es una sola columna se lee como serie y no es dataframe y por eso se usa .apply
datos[["cuota_deposito","cuota_limpieza"]]= datos[["cuota_deposito","cuota_limpieza"]].applymap(lambda x: x.replace("$","").replace(",","").strip())

<ipython-input-24-f494393b4709>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  datos[["cuota_deposito","cuota_limpieza"]]= datos[["cuota_deposito","cuota_limpieza"]].applymap(lambda x: x.replace("$","").replace(",","").strip())


In [25]:
datos[["cuota_deposito","cuota_limpieza"]]= datos[["cuota_deposito","cuota_limpieza"]].astype(np.float64)

In [26]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   evaluacion_general    3162 non-null   float64
 1   experiencia_local     3818 non-null   object 
 2   max_hospedes          3818 non-null   int64  
 3   descripcion_local     3818 non-null   object 
 4   descripcion_vecindad  3818 non-null   object 
 5   cantidad_baños        3818 non-null   int64  
 6   cantidad_cuartos      3818 non-null   int64  
 7   cantidad_camas        3818 non-null   int64  
 8   modelo_cama           3818 non-null   object 
 9   comodidades           3818 non-null   object 
 10  cuota_deposito        3818 non-null   float64
 11  cuota_limpieza        3818 non-null   float64
 12  precio                3818 non-null   float64
dtypes: float64(4), int64(4), object(5)
memory usage: 387.9+ KB


##**3. Datos de Texto**

In [27]:
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,This clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,Our century old Upper Queen Anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,Cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,Very lovely and cozy room for one. Convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,The “Studio at Mibbett Hollow' is in a Beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


In [28]:
#Recordando que cuando es una sola columna tenemos una series no un df
datos["descripcion_local"].head()

,descripcion_local
0,This clean and comfortable one bedroom sits ri...
1,Our century old Upper Queen Anne house is loca...
2,Cozy room in two-bedroom apartment along the l...
3,Very lovely and cozy room for one. Convenientl...
4,The “Studio at Mibbett Hollow' is in a Beautif...


In [29]:
#aqui usaremos varios metodos, .str es para convertir a string y .lower es para quitar las mayusculas y tener todo en minusculas
datos["descripcion_local"].str.lower()

,descripcion_local
0,this clean and comfortable one bedroom sits ri...
1,our century old upper queen anne house is loca...
2,cozy room in two-bedroom apartment along the l...
3,very lovely and cozy room for one. convenientl...
4,the “studio at mibbett hollow' is in a beautif...
...,...
3813,beautiful craftsman home in the historic wedgw...
3814,located in a very easily accessible area of se...
3815,this home is fully furnished and available wee...
3816,this business-themed modern home features: *h...


In [30]:
datos["descripcion_local"]=datos["descripcion_local"].str.lower()

In [31]:
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,this clean and comfortable one bedroom sits ri...,Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,our century old upper queen anne house is loca...,"Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,cozy room in two-bedroom apartment along the l...,The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,very lovely and cozy room for one. convenientl...,"Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,the “studio at mibbett hollow' is in a beautif...,--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


In [32]:
datos["descripcion_local"][3169]

"built, run and supported by seattle tech and start up veterans, grokhome's focus is to create a supportive environment for smart people working on interesting projects, start ups and more. this listing is an upper bunk, in a 2-person shared room. *note: this fall, there will be major renovations happening on one kitchen and bathroom at a time. there will always be two other working kitchens and two working bathrooms in the house. we'll work to minimize the impact these renovations have on your stay. **this listing is only available to those working in the tech/science space. live in a hacker house, and immerse yourself in the seattle tech scene. you can expect to be surrounded by smart people solving big problems or working on something fun. we have frequent demo nights, and love when our guests share something they are passionate about. if you're new to the city, our deep ties to the seattle tech scene can help you get involved. expand your network, develop your ideas, and learn some

In [33]:
#
datos["descripcion_local"]=datos["descripcion_local"].str.replace('[^a-zA-Z0-9\-\']',' ', regex=True)

In [34]:
datos["descripcion_local"]=datos["descripcion_local"].str.replace('(?<!\w)-(?<!\w)',' ', regex=True)

In [35]:
datos["descripcion_local"].head()

,descripcion_local
0,this clean and comfortable one bedroom sits ri...
1,our century old upper queen anne house is loca...
2,cozy room in two-bedroom apartment along the l...
3,very lovely and cozy room for one convenientl...
4,the studio at mibbett hollow' is in a beautif...


CREANDO LA TOQUENIZACION (PALABRA A PALABRA)

In [36]:
# una vez realizada la limpieza de todo tipo de caracteres innecesarios con REGEX, procedemos a la toquenizacion solamente usando .split()
datos["descripcion_local"]=datos["descripcion_local"].str.split()
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...",Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...",The convenience of being in Seattle but on the...,1,1,1,Futon,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"{Internet,""Wireless Internet"",Kitchen,""Free Pa...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",--,1,1,1,Real Bed,"{""Wireless Internet"",Kitchen,""Free Parking on ...",0.0,15.0,85.0


In [39]:
#si nos fijamos en la columna comodidades no se visualiza como una lista de tokens y la lista empieza con [], para ello hay que realizar el metodo .split
datos["comodidades"]=datos["comodidades"].str.replace('\{|}|\"','',regex=True)
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...",Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"Internet,Wireless Internet,Kitchen,Free Parkin...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"TV,Internet,Wireless Internet,Kitchen,Free Par...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...",The convenience of being in Seattle but on the...,1,1,1,Futon,"TV,Internet,Wireless Internet,Kitchen,Free Par...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"Internet,Wireless Internet,Kitchen,Free Parkin...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",--,1,1,1,Real Bed,"Wireless Internet,Kitchen,Free Parking on Prem...",0.0,15.0,85.0


In [40]:
datos["comodidades"]=datos["comodidades"].str.split(",")
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...",Lower Queen Anne is near the Seattle Center (s...,1,1,1,Real Bed,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","Upper Queen Anne is a really pleasant, unique ...",1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...",The convenience of being in Seattle but on the...,1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","Ballard is lovely, vibrant and one of the most...",1,1,1,Pull-out Sofa,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",--,1,1,1,Real Bed,"[Wireless Internet, Kitchen, Free Parking on P...",0.0,15.0,85.0


In [41]:
# Transformamos el texto en letras minúsculas
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.lower()
# Substituímos los caracteres especiales
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.replace('[^a-zA-Z0-9\-\']', ' ', regex=True)
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.replace('(?<!\w)-(?!\w)', '', regex=True)
# Transformamos el texto en lista, formando el token
datos['descripcion_vecindad'] = datos['descripcion_vecindad'].str.split()
datos.head()

,evaluacion_general,experiencia_local,max_hospedes,descripcion_local,descripcion_vecindad,cantidad_baños,cantidad_cuartos,cantidad_camas,modelo_cama,comodidades,cuota_deposito,cuota_limpieza,precio
0,10.0,--,1,"[this, clean, and, comfortable, one, bedroom, ...","[lower, queen, anne, is, near, the, seattle, c...",1,1,1,Real Bed,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,0.0,110.0
1,10.0,--,1,"[our, century, old, upper, queen, anne, house,...","[upper, queen, anne, is, a, really, pleasant, ...",1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,45.0
2,10.0,--,1,"[cozy, room, in, two-bedroom, apartment, along...","[the, convenience, of, being, in, seattle, but...",1,1,1,Futon,"[TV, Internet, Wireless Internet, Kitchen, Fre...",0.0,0.0,55.0
3,10.0,--,1,"[very, lovely, and, cozy, room, for, one, conv...","[ballard, is, lovely, vibrant, and, one, of, t...",1,1,1,Pull-out Sofa,"[Internet, Wireless Internet, Kitchen, Free Pa...",0.0,20.0,52.0
4,10.0,--,1,"[the, studio, at, mibbett, hollow', is, in, a,...",[],1,1,1,Real Bed,"[Wireless Internet, Kitchen, Free Parking on P...",0.0,15.0,85.0


#**DESAFIO 1**

Etapa 1

Proyecto Desafío 1: Ventas Online
El objetivo de este proyecto es analizar los resultados de un evento con los clientes de una empresa de venta online. Se recopiló un conjunto de datos que contiene los clientes que más gastaron en productos durante los 5 días de ventas, que es la duración del evento. Este análisis identificará al cliente con la mayor compra esta semana, quien recibirá un premio de la tienda, y posteriormente, puede ayudar a la empresa a crear nuevas estrategias para atraer más clientes.

La base de datos utilizada en este análisis es **dados_vendas_clientes.json** y contiene información importante sobre los clientes, como el nombre registrado del cliente, el monto total pagado al momento de la compra y el día de la compra.

Sabiendo esta información, el desafío del proyecto 1: ventas online será abrir la base de datos con Pandas y aplicar **json_normalize.**

In [ ]:
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_vendas_clientes.json')
# Aplicar json_normalize en la columna dados_vendas
datos = pd.json_normalize(datos['dados_vendas'])
# Mostrar valores
datos

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Así, el nuevo desafío del proyecto 1 será dividido en algunas metas:

**Eliminar datos en listas dentro del DataFrame;**

**Verificar tipos de datos;**

**Identificar columnas numéricas;**

**Transformar la columna numérica a tipo numérico.**

In [ ]:
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'Valor da compra'
datos['Valor da compra']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['Valor da compra'] = datos['Valor da compra'].apply(lambda x: x.replace('R$ ', '').replace(',','.').strip())
# Cambiar los tipo de datos para float
datos['Valor da compra'] = datos['Valor da compra'].astype(np.float64)
# Verificar la transformación
datos.info()

En el paso 2, trabajamos en la transformación de datos numéricos. Ahora podemos trabajar con valores textuales.

Debido a una inestabilidad en el sitio web de la empresa, tuvimos problemas con los nombres de los clientes durante el guardado. Esto resultó en una columna de nombres de clientes con una combinación de letras, mayúsculas y minúsculas, números y otros caracteres.

Sabiendo esto, manipula los textos de la columna **Cliente** para que el resultado sean los nombres de los clientes en letras minúsculas, con ausencia de caracteres especiales o números.

In [ ]:
# Transformar los textos de Cliente para texto en minúscula
datos['Cliente'] = datos['Cliente'].str.lower()
# Verificar el resultado
datos.head()

# Opción de substitución - necesario verificar el resultado de la substitución
# Regex no selecciona todas las letras de a-z y espacios en blanco ' '
# Todo que satisface el regex es borrado
datos['Cliente'].str.replace('[^a-z ]', '', regex=True)

# Realizar la substitución de los datos en la columna textual
datos['Cliente'] = datos['Cliente'].str.replace('[^a-z ]', '', regex=True).str.strip()
# Visualizar el resultado final
datos.head()

#**DESAFIO 2**

Administrar condominios es una tarea que requiere mucha atención y organización. Entre las diversas responsabilidades de gestión se encuentra el cobro del alquiler a los inquilinos. Para garantizar la buena salud financiera de la empresa, es fundamental que estos pagos se realicen de forma regular y puntual. Sin embargo, sabemos que esto no siempre sucede.

Teniendo esto en cuenta, propongo un desafío de procesamiento de datos con el objetivo de **analizar el retraso en el pago del alquiler en el condominio** de algunos residentes. Pongo a disposición la base de datos **dados_locacao_imoveis.json**, que contiene información sobre el departamento de los inquilinos, el día acordado para el pago del alquiler, el día en que se realiza el pago del alquiler y el monto del alquiler.

Con esta información, el desafío del proyecto 2: administración del condominio será similar al desafío del proyecto 1, abrir la base de datos con Pandas y aplicar **json_normalize** al DataFrame.

In [ ]:
# Import de pandas
import pandas as pd
# Leer el archivo json con read_json
datos = pd.read_json('/content/dados_locacao_imoveis.json')
# Aplicar json_normalize en la columna dados_locacao
datos = pd.json_normalize(datos['dados_locacao'])
# Mostrar valores
datos

Leímos la base de datos en el desafío anterior, ahora podemos seguir adelante con la transformación de estos datos. Entonces, de la misma manera que en el proyecto 1, el desafío del proyecto 2 está listado en algunas metas:

**Eliminar datos en listas dentro del DataFrame;**

**Verificar tipos de datos;**

**Identificar columnas numéricas;**

**Transformar la columna numérica a tipo numérico.**

In [ ]:
# Colectar los valores de las columnas y verificar
columnas = list(datos.columns)
columnas

# Destrincar las listas con explode
datos = datos.explode(columnas[1:])
# Resetear los index de las líneas
datos.reset_index(drop=True,inplace=True)
# Observar el DataFrame
datos

# Verificar los tipos de datos con info
datos.info()

# La columna numérica es el 'valor_aluguel'
datos['valor_aluguel']

# Iniciar la transformación
# Import de la biblioteca numpy
import numpy as np
# Remover los textos presentes en la base
# Cambiar las comas separadoras del decimal por punto
datos['valor_aluguel'] = datos['valor_aluguel'].apply(lambda x: x.replace('$ ', '').replace(' reais', '').replace(',','.').strip())
# Cambiar los tipos de datos para float
datos['valor_aluguel'] = datos['valor_aluguel'].astype(np.float64)
# Verificar la transformación
datos.info()

En el paso 2, trabajamos en transformar los datos numéricos. Ahora podemos trabajar con valores textuales.

Buscando explicar la organización de la identificación de los apartamentos, durante la creación del conjunto de datos se añadió el texto **(blocoAP)**. Este texto informa que los nombres de los apartamentos están organizados con la letra mayúscula seguida del número del apartamento. Sin embargo, esto no aporta ninguna información a nuestros datos, por lo que resulta interesante eliminar este texto del conjunto de datos.

Con esto, manipule los textos de la columna apartamento para eliminar el texto **(blocoAP)** del DataFrame.

In [ ]:
# Opción de substitución - necesario verificar el resultado de la substitución
# Fue necesario adicionar la barra '\' para ser considerados los paréntesis como caracteres literales
datos['apartamento'].str.replace(' \(blocoAP\)', '')

# Realizar la substitución de los datos en la columna textual
datos['apartamento'] = datos['apartamento'].str.replace(' \(blocoAP\)', '')
# Visualizar el resultado final
datos

#**Ejemplo 1**

Para realizar un estudio financiero de distribución salarial en una empresa es necesario transformar los datos relativos a los cargos y prepararlos para un estudio estadístico.

Por lo tanto, al leer el archivo con los datos, te encuentras con el siguiente DataFrame:

Selecciona la alternativa que hace que los datos de las columnas cantidad y salario se conviertan en valores numéricos, es decir, columnas int y float sin existencia de texto que acompañe a cada valor.

Para ayudarlo con sus pruebas, intente usar el siguiente código para crear el DataFrame df y encontrar la solución para la actividad:

In [ ]:
import pandas as pd
import numpy as np

# datos de la empresa
datos = {
    'cargo': ['Gerencia', 'Coordinación', 'Supervisión', 'Analista', 'Asistente', 'Operación', 'Asistente', 'Prácticas', 'Asesoría', 'Consultoría'],
    'cantidad': ['2 personas', '1 persona', '3 personas', '4 personas', '5 personas', '3 personas', '2 personas', '1 persona', '1 persona', '1 persona'],
    'salario': ['$10.000 reales', '$8.000 reales', '$7.000 reales', '$5.000 reales', '$4.000 reales', '$3.500 reales', '$3.000 reales', '$1.500 reales', '$2.500 reales', '$6.000 reales']
}

# transformando el diccionario en DataFrame
df = pd.DataFrame(datos)

# df es el DataFrame con los datos de la empresa
df

In [ ]:
#Eliminamos los textos que están alrededor de los números con apply y lambda.
df['cantidad'] = df['cantidad'].apply(lambda x: x.replace('personas', ''))
df['salario'] = df['salario'].apply(lambda x: x.replace('$', '').replace('reales', ''))

#Luego, transformamos las columnas a tipo numérico con astype.
df['cantidad'] = df['cantidad'].astype(np.int64)
df['salario'] = df['salario'].astype(np.float64)